In [ ]:
#импорты взяты с семинара
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
import torch.nn as nn
import cv2
from tqdm.notebook import tqdm
from torchvision.utils import save_image
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
%matplotlib inline

sns.set(style='darkgrid', font_scale=1.2)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!unzip /content/gdrive/MyDrive/ImageGenerationProject/datasets/edges2shoes_64-20210625T050510Z-001.zip -d /content/edges2shoes_64

Archive:  /content/gdrive/MyDrive/ImageGenerationProject/datasets/edges2shoes_64-20210625T050510Z-001.zip
replace /content/edges2shoes_64/edges2shoes_64/train/8577.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
DATA_SHOES_DIR = "/content/edges2shoes_64/edges2shoes_64"

In [ ]:
# взято с семинара
def get_dataloader(image_size, batch_size, dir):
  """
  Builds dataloader for training data.
  Use tt.Compose and tt.Resize for transformations
  :param image_size: height and wdith of the image
  :param batch_size: batch_size of the dataloader
  :returns: DataLoader object 
  """
  ds = ImageFolder(dir, transform=tt.Compose([
      #tt.Resize(image_size),
      #tt.CenterCrop(image_size),
      tt.ToTensor(),
      #tt.Normalize(*stats)
  ]))
  loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
  # TODO: resize images, convert them to tensors and build dataloader
  return loader

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(40, 25))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(images.detach()[:nmax], nrow=8).permute(1, 2, 0).cpu())

In [ ]:
image_size = 64
batch_size = 128

dataloader = get_dataloader((image_size, image_size * 2), batch_size, DATA_SHOES_DIR)
train_dl = DeviceDataLoader(dataloader, device)

In [ ]:
for j in dataloader:
    break

In [ ]:
print(j[0].shape)
print(j[1].shape)

torch.Size([128, 3, 64, 128])
torch.Size([128])


In [ ]:
for i in tqdm(train_dl):
    break

  0%|          | 0/68 [00:00<?, ?it/s]


In [ ]:
print(i[0].shape)

torch.Size([128, 3, 64, 128])


In [ ]:
for i in tqdm(train_dl):
    pass


100%|██████████| 68/68 [00:04<00:00, 15.61it/s]


In [ ]:
i[0].shape

torch.Size([4, 3, 64, 128])

In [ ]:
for i in tqdm(train_dl):
    print(i)

In [ ]:
def get_pair_images(img):
    first = img[:, :, :64]
    second = img[:, :, 64:]
    return (first, second)

In [ ]:
def get_pair_batch(img):
    first = img[:, :, :, :64]
    second = img[:, :, :, 64:]
    return (first, second)

In [ ]:
import tarfile

# open file
file = tarfile.open('/content/gdrive/MyDrive/ImageGenerationProject/datasets/edges2shoes.tar.gz')
  
# extracting file
file.extractall('/content')
  
file.close()

In [ ]:
class Block(nn.Module):
  def __init__(self, in_channels, out_channels, stride):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=4, stride=stride, padding=1, padding_mode="reflect"),
        nn.InstanceNorm2d(num_features=out_channels),
        nn.LeakyReLU(0.2)
    )

  def forward(self, x):
    return self.conv(x)

In [ ]:
class Discriminator(nn.Module):
  def __init__(self, in_channels=3, features = [64, 128, 256, 512]):
    super().__init__()
    self.initial = nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=features[0], kernel_size=4, stride=2, padding=1, padding_mode="reflect"),
        nn.LeakyReLU(0.2)
    )

    layers = []
    in_channels = features[0]
    for feature in features[1:]:
      layers.append(Block(in_channels=in_channels, out_channels=feature, stride = 2 if feature != features[-1] else 1))
      in_channels=feature
    layers.append(nn.Conv2d(in_channels=in_channels, out_channels=1, kernel_size=6, stride=2, padding_mode="reflect"))
    self.lay = layers
    self.model = nn.Sequential(*layers)

  def forward(self, x):
    x = self.initial(x)
    
    return torch.sigmoid(self.model(x))
t = torch.randn(2, 3, 64, 64)
mod = Discriminator()
out = mod(t).view(-1, 1)
out.shape

torch.Size([2, 1])

In [ ]:
class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, padding_mode="reflect", **kwargs)
        if down else nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels, **kwargs),
        nn.InstanceNorm2d(num_features=out_channels),
        nn.ReLU(inplace=True) if use_act else nn.Identity()
    )

  def forward(self, x):
    return self.conv(x)


In [ ]:
class ResidualBlock(nn.Module):
  def __init__(self, channels):
    super().__init__()
    self.block = nn.Sequential(
        ConvBlock(channels, channels, kernel_size=3, padding=1),
        ConvBlock(channels, channels, use_act=False, kernel_size=3, padding=1),
    )
  def forward(self, x):
    x = self.block(x)
    return x

In [ ]:
class Generator(nn.Module):
  def __init__(self, img_channels, num_features=64, num_residuals=7):
    super().__init__()
    self.initial = nn.Sequential(
        nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
        nn.ReLU(inplace=True)
    )

    self.down_blocks = nn.ModuleList(
        [
          ConvBlock(num_features, num_features*2, kernel_size=3, stride=2, padding=1),
          ConvBlock(num_features*2, num_features*4, kernel_size=3, stride=2, padding=1),
        ]
    )

    self.residual_blocks = nn.Sequential(
        *[ResidualBlock(num_features*4) for _ in range(num_residuals)]
    )

    self.up_blocks = nn.ModuleList(
        [
        ConvBlock(num_features*4, num_features*2, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
        ConvBlock(num_features*2, num_features, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
        ]
    )

    self.last = nn.Conv2d(num_features, img_channels, kernel_size=7, stride=1, padding=3, padding_mode="reflect")

  def forward(self, x):
    x = self.initial(x)
    for lay in self.down_blocks:
      x = lay(x)
    x = self.residual_blocks(x)
    for lay in self.up_blocks:
      x = lay(x)

    return torch.tanh(self.last(x))
t1 = torch.randn(1, 3, 64, 64)
g = Generator(3)
print(g(t1).shape)

torch.Size([1, 3, 64, 64])


In [ ]:
generator = Generator(3).to(device)

In [ ]:
discriminator = Discriminator().to(device)

In [ ]:
discriminator.load_state_dict(torch.load("/content/gdrive/MyDrive/ImageGenerationProject/train/models/epoch#_112_discriminator"))

<All keys matched successfully>

In [ ]:
generator.load_state_dict(torch.load("/content/gdrive/MyDrive/ImageGenerationProject/train/models/epoch#_112_generator"))

<All keys matched successfully>

In [ ]:
lr = 0.0001#0.0001

model = {
    "discriminator": discriminator,
    "generator": generator
}

criterion = {
    "discriminator": nn.BCELoss(),
    "generator": nn.BCELoss()
}

In [ ]:
def save_models(num_epoch):
  path = "/content/gdrive/MyDrive/ImageGenerationProject/train/models/epoch#_{0}_{1}"
  torch.save(model['discriminator'].state_dict(), path.format(num_epoch, 'discriminator'))
  torch.save(model['generator'].state_dict(), path.format(num_epoch, 'generator'))

In [ ]:
def save_samples(index, latent_tensors, show=True):
    path = "/content/gdrive/MyDrive/ImageGenerationProject/train/pictures"
    fake_images = model["generator"](latent_tensors)
    fake_fname = 'generated-images-{0:0=4d}.png'.format(index)
    save_image(fake_images, os.path.join(path, fake_fname), nrow=2)
    print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))
        fake_images = fake_images.cpu().detach()
        print(fake_images.shape)
        for i in range(2):
          for j in range(2):
            ax[i][j].set_xticks([]); ax[i][j].set_yticks([])
            ax[i][j].imshow(fake_images[i * 2 + j].permute(1, 2, 0))

In [ ]:
def show_batch(dl, nmax=64):
    for images in dl:
        show_images(images, nmax)

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
# взят из семинара
def fit(model, criterion, epochs, lr, start_idx=1):
    model["discriminator"].train()
    model["generator"].train()
    torch.cuda.empty_cache()

    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []

    # Create optimizers
    optimizer = {
          "discriminator": torch.optim.Adam(model["discriminator"].parameters(), 
                                            lr=lr, betas=(0.5, 0.999)),
          "generator": torch.optim.Adam(model["generator"].parameters(),
                                        lr=lr, betas=(0.5, 0.999))
    }
    

    for epoch in range(epochs):
        loss_d_per_epoch = []
        loss_g_per_epoch = []
        real_score_per_epoch = []
        fake_score_per_epoch = []
        for batch, _ in tqdm_notebook(train_dl):
            edges, imgs = get_pair_batch(batch)
            # Train discriminator
            # Clear discriminator gradients
            optimizer["discriminator"].zero_grad()

            # Pass real images through discriminator
            real_preds = model["discriminator"](imgs).view(-1, 1)
            real_targets = torch.ones(imgs.size(0), 1, device=device)
            real_loss = criterion["discriminator"](real_preds, real_targets)
            cur_real_score = torch.mean(real_preds).item()
            
            # Generate fake images
            ### latent =  torch.randn(batch_size, latent_size, 1, 1, device=device)
            fake_images = model["generator"](edges)

            # Pass fake images through discriminator
            fake_targets = torch.zeros(fake_images.size(0), 1, device=device)
            fake_preds = model["discriminator"](fake_images).view(-1, 1)
            fake_loss = criterion["discriminator"](fake_preds, fake_targets)
            cur_fake_score = torch.mean(fake_preds).item()

            real_score_per_epoch.append(cur_real_score)
            fake_score_per_epoch.append(cur_fake_score)

            # Update discriminator weights
            loss_d = real_loss + fake_loss
            loss_d.backward()
            optimizer["discriminator"].step()
            loss_d_per_epoch.append(loss_d.item())


            # Train generator
            # Clear generator gradients
            optimizer["generator"].zero_grad()
            
            # Generate fake images
            fake_images = model["generator"](edges)
            
            # Try to fool the discriminator
            
            preds = model["discriminator"](fake_images).view(-1, 1)
            targets = torch.ones(preds.shape[0], 1, device=device)

            #print('preds_size = {0}, targets_size = {1}, fake_images_size={2}, edges_size={3}, img_size={4}'.format(preds.shape, targets.shape, fake_images.shape, edges.shape, imgs.shape))
            loss_g = criterion["generator"](preds, targets)
            
            # Update generator weights
            loss_g.backward()
            optimizer["generator"].step()
            loss_g_per_epoch.append(loss_g.item())
        # Save model
        save_models(epoch + start_idx)
        # Record losses & scores
        losses_g.append(np.mean(loss_g_per_epoch))
        losses_d.append(np.mean(loss_d_per_epoch))
        real_scores.append(np.mean(real_score_per_epoch))
        fake_scores.append(np.mean(fake_score_per_epoch))
        
        # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
            epoch+1, epochs, 
            losses_g[-1], losses_d[-1], real_scores[-1], fake_scores[-1]))
    
        # Save generated images
        #if epoch == epochs - 1:
        save_samples(epoch+start_idx, edges[:4], show=True)
    
    return losses_g, losses_d, real_scores, fake_scores

In [ ]:
epochs = 400
fit(model, criterion, epochs, lr, 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Epoch [1/400], loss_g: 1.1384, loss_d: 1.6263, real_score: 0.4612, fake_score: 0.5355
Saving generated-images-0000.png


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


torch.Size([4, 3, 64, 64])



Epoch [2/400], loss_g: 1.0265, loss_d: 1.5084, real_score: 0.4789, fake_score: 0.5185
Saving generated-images-0001.png


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


torch.Size([4, 3, 64, 64])



Epoch [85/400], loss_g: 2.1837, loss_d: 0.8432, real_score: 0.7084, fake_score: 0.2823


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Saving generated-images-0084.png
torch.Size([4, 3, 64, 64])
